# File ingestion and schema validation

> [dask]('https://www.dask.org/')

Try different methods of file reading eg: Dask, Modin, Ray, pandas

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

Create Schema in YAML

Validate the file with YAML

Write pipe separated text file | in gz format and create a summary of the file.

In [1]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55969,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.73 GiB
Comm: tcp://127.0.0.1:55999,Total threads: 3
Dashboard: http://127.0.0.1:56002/status,Memory: 3.93 GiB
Nanny: tcp://127.0.0.1:55975,


In [2]:
#client.shutdown()

In [3]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import csv

> YAML Configuration file write and read

In [4]:
%%writefile file.yaml
file_type: csv
dataset_name: Parking_Violations
file_name: Parking_Violations_2016
inbound_deliminater: ','
outbound_deliminater: '|'
skip_leading_rows: 1
columns:
   - summons_number 
   - plate_id 
   - registration_state
   - plate_type      
   - issue_date
   - violation_code 
   - vehicle_body_type
   - vehicle_make      
   - issuing_agency
   - street_code1
   - street_code2
   - street_code3       
   - vehicle_expiration_date 
   - violation_location
   - violation_precinct
   - issuer_precinct
   - issuer_code 
   - issuer_command 
   - issuer_squad
   - violation_time 
   - time_first_observed 
   - violation_county
   - violation_in_front_of_or_opposite
   - house_number
   - street_name 
   - intersecting_street 
   - date_first_observed 
   - law_section
   - sub_division 
   - violation_legal_code
   - days_parking_in_effect
   - from_hours_in_effect 
   - to_hours_in_effect
   - vehicle_color
   - unregistered_vehicle
   - vehicle_year
   - meter_number
   - feet_from_curb 
   - violation_post_code 
   - violation_description
   - no_standing_or_stopping_violation 
   - hydrant_violation
   - double_parking_violation

Overwriting file.yaml


In [5]:
import yaml
with open('file.yaml', 'r') as f:
    file = yaml.safe_load(f)
    file

In [6]:
file['inbound_deliminater']

','

In [ ]:
file['columns']

> Read and inspect CSV file

In [ ]:
import dask.dataframe as dd
dfd = dd.read_csv('Parking_Violations_2015.csv', delimiter=',')
dfd

In [ ]:
dfd.dtypes

### Perform Basic Validation on Data Columns

- Validate csv file with yaml

Drop unwanted columns

In [10]:
dfd=dfd.drop(["BIN", "BBL", "NTA"], axis=1)
dfd=dfd.drop(["Latitude", "Longitude", "Community Board", "Community Council ", "Census Tract"], axis=1)

> Remove special characters and white space from data columns

In [11]:
def val_data_col():
    # clean up df columns #
    dfd.columns=dfd.columns.str.replace('[?]', '')
    dfd.columns=dfd.columns.str.strip()
    dfd.columns=dfd.columns.str.replace('[ ]', '_')
    dfd.columns=dfd.columns.str.lower()
    # compare yaml columns with df columns #
    expected_columns = list(file['columns'])
    if len(dfd.columns) == len(expected_columns) and list(dfd.columns) == expected_columns:
        print('column name and column length validation passed')
        mismatched_columns_file = list(set(dfd.columns).difference(expected_columns))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_columns).difference(dfd.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {dfd.columns}')
        logging.info(f'expected columns: {expected_columns}')
        return 1
    else:
        print('column name and column length validation failed')

    return 0

In [12]:
val_data_col()

column name and column length validation passed
Following File columns are not in the YAML file []
Following YAML columns are not in the file uploaded []


C:\Users\nunto\AppData\Local\Temp\ipykernel_30044\630901633.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd.columns=dfd.columns.str.replace('[?]', '')
C:\Users\nunto\AppData\Local\Temp\ipykernel_30044\630901633.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd.columns=dfd.columns.str.replace('[ ]', '_')


1

In [ ]:
dfd.dtypes

Save new dataframe

In [ ]:
dfd.to_csv('validated_file/export-*.csv')

Use yaml file to validate